# `apps_long.csv`

In [50]:
library(dplyr)
library(readr)
library(tidyr)

cr_apps <- read_csv("../output/apps_long.csv", show_col_types = FALSE)

pl_ssns <- read_csv("../output/player_ssns.csv", show_col_types = FALSE)

gls <- read_csv("../output/scorers-long.csv", show_col_types = FALSE) %>%
    group_by(
        season,
        game_no,
        player_name
    ) %>%
    summarise(
        goals_scored = sum(goals_scored),
        .groups = "drop"
    )

res <- read_csv("https://raw.githubusercontent.com/petebrown/update-results/main/data/results_df.csv", show_col_types = FALSE)

In [51]:
library(stringr)

sb_game_ids <- read_csv("https://raw.githubusercontent.com/petebrown/update-player-stats/main/data/players_df.csv", show_col_types = FALSE) %>%
    select(
        sb_game_id,
        game_date        
    ) %>%
    rename(
        game_id = sb_game_id
    ) %>%
    mutate(
        game_id = str_remove(game_id, "tpg"),
        game_id = as.numeric(game_id)
    ) %>%
    unique() %>%
    arrange(
        game_date
    )

head(sb_game_ids, 3)

game_id,game_date
<dbl>,<date>
240309,1996-08-17
243200,1996-08-21
242383,1996-08-23


#### Pivot apps_long to wide format mimicking book

In [55]:
apps_long <- cr_apps %>%
    left_join(
        pl_ssns %>%
            select(
                season,
                disam_name,
                player_name
            ) %>% unique(),
        by = c("season" = "season", "player_name" = "disam_name")
    ) %>%
    mutate(
        player_name = player_name.y
    ) %>%
    select(-player_name.y) %>%
    left_join(
        gls,
        by = c("season" = "season", "game_no" = "game_no", "player_name" = "player_name")
    ) %>%
    left_join(
        res %>%
            select(
                season,
                ssn_game_no,
                generic_comp
            ),
        by = c("season" = "season", "game_no" = "ssn_game_no")
    ) %>%
    group_by(
        season,
        player_name,
        generic_comp,
        role
    ) %>%
    summarise(
        count = n()
    )



    # %>%
    # arrange(
    #     season,
    #     game_no,
    #     player_name
    # ) %>%
    # pivot_wider(
    #     id_cols = c(season, game_no),
    #     names_from = player_name,
    #     values_from = shirt_no
    # ) %>%
    
    # select(
    #     season, game_no, opposition, score, generic_comp,
    #     sort(names(.[3:length(.)]))
    # )

`summarise()` has grouped output by 'season', 'player_name', 'generic_comp'.
You can override using the `.groups` argument.


In [56]:
gls_long <- cr_apps %>%
    left_join(
        pl_ssns %>%
            select(
                season,
                disam_name,
                player_name
            ) %>% unique(),
        by = c("season" = "season", "player_name" = "disam_name")
    ) %>%
    mutate(
        player_name = player_name.y
    ) %>%
    select(-player_name.y) %>%
    left_join(
        gls %>%
            select(
                season,
                game_no,
                player_name,
                goals_scored
            ),
        by = c("season", "game_no", "player_name")
    ) %>%
    select(
        season,
        game_no,
        player_name,,
        goals_scored
    ) %>%
    left_join(
        res %>%
            select(
                season,
                ssn_game_no,
                generic_comp
            ),
        by = c("season" = "season", "game_no" = "ssn_game_no")
    ) %>%
    mutate(
        goals_scored = ifelse(is.na(goals_scored), 0, goals_scored)
    ) %>%
    group_by(
        season,
        player_name,
        generic_comp
    ) %>%
    summarise(
        count = sum(goals_scored)
    ) %>%
    mutate(
        role = "goals"
    )

head(apps_long, 3)

`summarise()` has grouped output by 'season', 'player_name'. You can override
using the `.groups` argument.


season,player_name,generic_comp,role,count
<chr>,<chr>,<chr>,<chr>,<int>
1989/90,Chris Malkin,Associate Members' Cup,starter,5
1989/90,Chris Malkin,Associate Members' Cup,sub,1
1989/90,Chris Malkin,FA Cup,sub,1


In [61]:
rbind(apps_long, gls_long) %>%
    mutate(
        generic_comp = ifelse(generic_comp %in% c("Football League", "Non-League"), "League", generic_comp)
    ) %>%
    filter(
        season == "1989/90"
    ) %>%
    mutate(
        generic_comp = factor(
            generic_comp,
            levels = c("League", "FA Cup", "League Cup", "Associate Members' Cup", "Anglo-Italian Cup", "Full Members' Cup", "FA Trophy")
        ),
        role = factor(
            role,
            levels = c("starter", "sub", "goals")
        )
    ) %>%
    left_join(
        pl_ssns %>% select(season, player_name, disam_name) %>% unique(),
        by = c("season" = "season", "player_name" = "player_name")
    ) %>%
    arrange(
        season,
        disam_name
    ) %>%
    pivot_wider(
        id_cols = c(season, generic_comp, role),
        names_from = disam_name,
        values_from = count,
        values_fill = 0
    ) %>%
    arrange(
        season,
        generic_comp,
        role
    )

season,generic_comp,role,Bauress,Bishop,Fairclough,Garnett,Harvey,Higgins,Hughes,⋯,McCarrick,McNab,Morrissey,Muir,Mungall,Nixon,Pike,Steel,Thomas,Vickers
<chr>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1989/90,League,starter,1,13,3,5,48,47,48,⋯,32,25,24,49,19,49,2,32,45,45
1989/90,League,sub,0,16,12,0,1,0,0,⋯,0,0,3,0,1,0,0,7,0,0
1989/90,League,goals,0,7,1,0,7,1,4,⋯,4,1,5,24,2,0,0,5,2,3
1989/90,FA Cup,starter,0,1,0,0,1,1,1,⋯,0,0,1,1,1,1,1,1,0,1
1989/90,FA Cup,sub,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
1989/90,FA Cup,goals,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
1989/90,League Cup,starter,0,4,0,1,7,7,7,⋯,5,0,6,7,2,7,2,3,5,5
1989/90,League Cup,sub,0,3,3,0,0,0,0,⋯,0,0,0,0,0,0,0,2,0,0
1989/90,League Cup,goals,0,2,0,0,0,0,0,⋯,1,0,0,4,0,0,0,2,0,1


In [41]:
res %>% filter(season > 1988) %>% select(generic_comp) %>% unique()

generic_comp
<chr>
Football League
Associate Members' Cup
FA Cup
League Cup
Non-League
FA Trophy
Anglo-Italian Cup
Full Members' Cup
